In [4]:
import os
import librosa
import numpy as np
import pandas as pd
from pycaret.classification import *
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, mean_squared_error, r2_score)
from sklearn.model_selection import cross_val_score, train_test_split

In [5]:
# Caminhos para as pastas de áudio
pasta_ruido = r'../audios/audios_treinamento/classe_ruido'
pasta_barco = r'../audios/audios_treinamento/classe_barco'

# Função para extrair features dos áudios
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    features = {
        'chroma_stft': librosa.feature.chroma_stft(y=y, sr=sr).mean(),
        'chroma_cqt': librosa.feature.chroma_cqt(y=y, sr=sr).mean(),
        'chroma_cens': librosa.feature.chroma_cens(y=y, sr=sr).mean(),
        'rmse': librosa.feature.rms(y=y).mean(),
        'spectral_centroid': librosa.feature.spectral_centroid(y=y, sr=sr).mean(),
        'spectral_bandwidth': librosa.feature.spectral_bandwidth(y=y, sr=sr).mean(),
        'spectral_contrast': librosa.feature.spectral_contrast(y=y, sr=sr).mean(),
        'spectral_flatness': librosa.feature.spectral_flatness(y=y).mean(),
        'spectral_rolloff': librosa.feature.spectral_rolloff(y=y, sr=sr).mean(),
        'rolloff': librosa.feature.spectral_rolloff(y=y, sr=sr).mean(),
        'zero_crossing_rate': librosa.feature.zero_crossing_rate(y).mean(),
        'mfcc1': librosa.feature.mfcc(y=y, sr=sr)[0].mean(),
        'mfcc2': librosa.feature.mfcc(y=y, sr=sr)[1].mean(),
        'mfcc3': librosa.feature.mfcc(y=y, sr=sr)[2].mean(),
        # 'mfcc4': librosa.feature.mfcc(y=y, sr=sr)[3].mean(),
        # 'mfcc5': librosa.feature.mfcc(y=y, sr=sr)[4].mean(),
        # 'mfcc6': librosa.feature.mfcc(y=y, sr=sr)[5].mean(),
        # 'mfcc7': librosa.feature.mfcc(y=y, sr=sr)[6].mean(),
        # 'mfcc8': librosa.feature.mfcc(y=y, sr=sr)[7].mean(),
        # 'mfcc9': librosa.feature.mfcc(y=y, sr=sr)[8].mean(),
        # 'mfcc10': librosa.feature.mfcc(y=y, sr=sr)[9].mean(),
        # 'mfcc11': librosa.feature.mfcc(y=y, sr=sr)[10].mean(),
        # 'mfcc12': librosa.feature.mfcc(y=y, sr=sr)[11].mean(),
        # 'mfcc13': librosa.feature.mfcc(y=y, sr=sr)[12].mean(),
        #'tempo': librosa.beat.tempo(y=y, sr=sr)[0]
    }
    return features

# Função para processar as pastas e adicionar os dados ao DataFrame
def verifica_pastas(folder_path, label):
    data = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_path.endswith('.wav'):
            features = extract_features(file_path)
            features['label'] = label
            data.append(features)
    return data

# Processar as duas pastas sendo cada uma uma classe
dados_ruido = verifica_pastas(pasta_ruido, 1)
dados_barco = verifica_pastas(pasta_barco, 2)

# Combinar os dados e criar um DataFrame
data = dados_ruido + dados_barco
df = pd.DataFrame(data)

# Salvar em um arquivo CSV
df.to_csv("csv/features_audios_classificados.csv", index=False)

In [6]:
dados = pd.read_csv("csv/features_audios_classificados.csv")
X = dados.drop(columns=['label'])
y = dados['label']

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Treinando o modelo LDA
lr_model = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
lr_model.fit(X_train, y_train)

# Fazer previsões e avaliar o modelo
y_pred = lr_model.predict(X_test)
print("Relatório de Classificação para LDA:")
print(classification_report(y_test, y_pred))

clf1 = setup(data=dados, target='label', session_id=123)
save_model(lr_model, 'Modelo_LDA')

Relatório de Classificação para LDA:
              precision    recall  f1-score   support

           1       1.00      0.92      0.96        13
           2       0.95      1.00      0.98        20

    accuracy                           0.97        33
   macro avg       0.98      0.96      0.97        33
weighted avg       0.97      0.97      0.97        33



,Description,Value
0,Session id,123
1,Target,label
2,Target type,Binary
3,Target mapping,"1: 0, 2: 1"
4,Original data shape,"(66, 15)"
5,Transformed data shape,"(66, 15)"
6,Transformed train set shape,"(46, 15)"
7,Transformed test set shape,"(20, 15)"
8,Numeric features,14
9,Preprocess,True


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['chroma_stft', 'chroma_cqt',
                                              'chroma_cens', 'rmse',
                                              'spectral_centroid',
                                              'spectral_bandwidth',
                                              'spectral_contrast',
                                              'spectral_flatness',
                                              'spectral_r...
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                    

In [7]:


scores = cross_val_score(lr_model, X, y, cv=5)
print(f"Cross-validation scores: {scores}")
print(f"Mean cross-validation score: {np.mean(scores)}")

Cross-validation scores: [0.85714286 1.         0.84615385 1.         0.92307692]
Mean cross-validation score: 0.9252747252747253
